## 配置环境

In [3]:
# 安装git lfs
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.


In [5]:
# 从huggingface上拉取模型
!pwd
!cd .. && git lfs install && git clone https://huggingface.co/THUDM/chatglm-6b-int4

/root/autodl-tmp/model/knowledge_base/ChatRats/ptuning
/root/autodl-tmp/model/knowledge_base/ChatRats/ptuning


In [ ]:
# 安装cuda
!apt install nvidia-cuda-toolkit

In [8]:
# 配置python环境
!conda activate NLPpython
%pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


## 进行微调

In [9]:
PRE_SEQ_LEN=128
LR=2e-2

!CUDA_VISIBLE_DEVICES=0

!cd utils/ && python3 main.py \
    --do_train \
    --train_file ../../data/merged.json \
    --validation_file ../../data/val_set.json \
    --prompt_column question \
    --response_column answer \
    --overwrite_cache \
    --model_name_or_path ../../chatglm-6b-int4 \
    --output_dir output/adgen-chatglm-6b-pt-$PRE_SEQ_LEN-$LR \
    --overwrite_output_dir \
    --max_source_length 128 \
    --max_target_length 128 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 16 \
    --predict_with_generate \
    --max_steps 6000 \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate $LR \
    --pre_seq_len $PRE_SEQ_LEN \
    --quantization_bit 4


/root/autodl-tmp/model/knowledge_base/ChatRats/ptuning/utils


## 微调评估

In [ ]:
PRE_SEQ_LEN=128
CHECKPOINT="adgen-chatglm-6b-pt-128-2e-2"
STEP=6000

!CUDA_VISIBLE_DEVICES=0
!cd utils/ && python3 main.py \
    --do_predict \
    --validation_file ../../val_set.json \
    --test_file ../../test_set.json \
    --overwrite_cache \
    --prompt_column question \
    --response_column answer \
    --model_name_or_path ../../chatglm-6b-int4 \
    --ptuning_checkpoint output/$CHECKPOINT/checkpoint-$STEP \
    --output_dir output/$CHECKPOINT \
    --overwrite_output_dir \
    --max_source_length 128 \
    --max_target_length 128 \
    --per_device_eval_batch_size 1 \
    --predict_with_generate \
    --pre_seq_len $PRE_SEQ_LEN \
    --quantization_bit 4
